In [45]:
import os
import pandas as pd
import numpy as np

data=pd.DataFrame(columns=['age','gender','race','location'])
gender={'male':0,'female':1}
race={'White':0, 'Black':1, 'Asian':2, 'Indian':3, 'Others':4 }

cnt=0
for a,b,c in os.walk('UTKFace'):
        for j in c:
            try:
                l=j.split('_')
                l.pop()
                l.append('UTKFace/'+j)
                s=pd.Series(l,index=data.columns)
                data=data.append(s,ignore_index=True)
            except:
                continue

In [46]:
data.head()

,age,gender,race,location
0,9,1,2,UTKFace/9_1_2_20161219204347420.jpg.chip.jpg
1,36,0,1,UTKFace/36_0_1_20170117163203851.jpg.chip.jpg
2,86,1,0,UTKFace/86_1_0_20170120225751953.jpg.chip.jpg
3,26,1,0,UTKFace/26_1_0_20170116171048641.jpg.chip.jpg
4,1,1,2,UTKFace/1_1_2_20161219154612988.jpg.chip.jpg


In [71]:
from sklearn.model_selection import train_test_split

X=data['location']
y=data.drop('location',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

In [74]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Dense,BatchNormalization,MaxPool2D,Activation

class model():
    
    def common_model(self,inp_shape):
    
        x=Con2D(64,(3,3),Activation='relu')(inp_shape)
        x=MaxPool2D((2,2))(x)
        x=BatchNormalization(ax(x)

        x=Con2D(64,(3,3),Activation='relu')(x)
        x=MaxPool2D((2,2))(x)
        x=BatchNormalization(x)

        x=Con2D(64,(3,3),Activation='relu')(x)
        x=MaxPool2D((2,2))(x)
        x=BatchNormalization(x)

        x=Con2D(64,(3,3),Activation='relu')(x)
        x=MaxPool2D((2,2))(x)
        x=BatchNormalization(x)

        return x

    def race_model():
            
            
            
    def age_model():
        
    def gender_model():

    def assemble_model(self,inp_shape):
                             
        

SyntaxError: invalid syntax (<ipython-input-74-47acee3d1585>, line 13)